<a href="https://colab.research.google.com/github/aebonlee/chatbot_01team/blob/main/ICT_%EB%B9%84%EC%A0%84%EA%B3%B5%EC%9E%901%EC%A1%B0_ver_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-openai tiktoken python-dotenv
#!pip install orjson
#!pip install gradio==4.*
#!pip install trafilatura

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
import os, json
from typing import List, Dict, Any
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

In [ ]:
멋진 UI 전

In [ ]:
# API키 및 모델 설정
os.environ["OPENAI_API_KEY"] = ""
OPENAI_MODEL = "gpt-4o-mini"

MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
llm_g = ChatOpenAI(model=MODEL, temperature=0)     # 가드/판정/리라이트
llm_x = ChatOpenAI(model=MODEL, temperature=0.2)   # 자체 추출
llm_s = ChatOpenAI(model=MODEL, temperature=0.2)   # 요약

KNOWLEDGE_CUTOFF = "2024-06"

STYLE_SYS = (
    "한국어 존댓말을 사용합니다. 말투는 친절하고 전문적으로 유지합니다. "
    "핵심은 간결하게 전달하되, 전력·에너지·모빌리티 분야의 수치/단위/기호(η, THD, pf, pu, kW, kWh, °C 등)는 보존합니다. "
    "불확실하거나 기억이 모호한 내용은 '불확실'로 표시하고 추정·일반론은 명확히 구분합니다. 과장 표현은 지양합니다."
)

ALLOW = [
    "전력","에너지","모빌리티","EV","충전","배터리","신재생","태양광","풍력","수소",
    "인버터","컨버터","PCS","BESS","V2G","EMS","DER","DR","스마트그리드","송배전","변전",
    "THD","pf","pu","고조파","전압안정도","신뢰도","시장","트렌드","동향"
]

def jload(s: str, default: Any) -> Any:
    try:
        return json.loads(s)
    except:
        return default

def evaluate_answerability(q: str) -> Dict[str, Any]:
    """
    JSON 예시
    {
      "answerable": true|false,
      "reasons": ["off_topic","requires_web","knowledge_cutoff","ambiguous","too_broad","privacy_sensitive","unsafe"],
      "explain": "왜 바로 답하기 어려운지 한 줄 설명(존댓말)",
      "ask_for": ["누락된 구체 정보 항목들"],
      "rewrite_examples": ["권장 재질문 1","권장 재질문 2"]
    }
    """
    policy = (
        "당신은 질의가 아래 제약에서 답변 가능한지 판정합니다.\n"
        f"- 지식 컷오프: {KNOWLEDGE_CUTOFF}\n"
        "- 외부 웹 검색/실시간 데이터 사용 불가\n"
        "- 도메인: 전력·에너지·모빌리티 중심\n"
        "- 개인식별/민감정보, 불법/위험, 의료·법률·투자 개별 조언은 불가\n"
        "- 모호/과도하게 광범위/정의가 불명확하면 구체화가 필요\n"
        "위 조건을 바탕으로 JSON만 반환하세요."
    )
    schema = {
        "answerable": True,
        "reasons": [],
        "explain": "",
        "ask_for": [],
        "rewrite_examples": []
    }
    prompt = (
        policy + "\n\n"
        "가능하면 사용자가 바로 쓸 수 있는 '재질문 예시'를 1~3개 제안하세요 "
        "(전력·에너지 맥락의 지표/범위/지역/기간/대상 등을 포함). "
        "JSON만 출력. 스키마는 다음과 같음:\n"
        + json.dumps(schema, ensure_ascii=False) + "\n\n"
        f"질문: {q}"
    )
    out = llm_g.invoke([
        SystemMessage(content=STYLE_SYS),
        SystemMessage(content="JSON only."),
        HumanMessage(content=prompt)
    ]).content
    data = jload(out, schema)
    # 휴리스틱 보강: 실시간/최신/가격/주가/날씨 등
    recent_triggers = ["실시간","현재","오늘","방금","지금","최신","주가","환율","날씨","스코어","속보","라이브"]
    if any(t in q for t in recent_triggers):
        data["answerable"] = False
        if "requires_web" not in data["reasons"]:
            data["reasons"].append("requires_web")
        data["explain"] = data.get("explain") or "실시간·최신 데이터는 웹 접근 없이 정확히 안내드리기 어렵습니다."
    # 도메인 체크
    if not any(k.lower() in q.lower() for k in ALLOW):
        if "off_topic" not in data["reasons"]:
            data["reasons"].append("off_topic")
    return data

def guidance_message(evald: Dict[str, Any]) -> str:
    reasons_ko = {
        "off_topic": "준비된 주제 범위를 벗어났습니다.",
        "requires_web": "실시간/웹 검색이 필요한 주제입니다.",
        "knowledge_cutoff": f"지식 컷오프({KNOWLEDGE_CUTOFF}) 이후 정보가 필요합니다.",
        "ambiguous": "질문이 모호합니다.",
        "too_broad": "질문 범위가 너무 넓습니다.",
        "privacy_sensitive": "개인정보·민감정보라서 도와드리기 어렵습니다.",
        "unsafe": "위험/부적절한 요청입니다."
    }
    lines = ["요청하신 내용은 바로 정확히 답변드리기 어렵습니다."]
    if evald.get("explain"):
        lines.append(f"사유: {evald['explain']}")
    if evald.get("reasons"):
        labs = [reasons_ko.get(r, r) for r in evald["reasons"]]
        lines.append("판정: " + ", ".join(labs))
    if evald.get("ask_for"):
        lines.append("\n다음 정보를 알려주시면 더 정확히 도와드릴 수 있습니다:")
        lines += [f"- {a}" for a in evald["ask_for"][:6]]
    if evald.get("rewrite_examples"):
        lines.append("\n이렇게 질문해 보시면 좋습니다:")
        lines += [f"- {r}" for r in evald["rewrite_examples"][:3]]
    return "\n".join(lines)

def make_subquestions(q: str) -> List[str]:
    prompt = (
        "다음 질문을 3~5개의 하위 질문으로 분해하세요. 최신성 이슈가 있으면 '최근 동향 확인'을 포함합니다. "
        "JSON만 출력: {\"subs\": [\"...\"]}\n\n"
        f"질문: {q}"
    )
    out = llm_x.invoke([SystemMessage(content="JSON only."), HumanMessage(content=prompt)]).content
    data = jload(out, {"subs":[q]})
    subs = [s for s in data.get("subs", []) if isinstance(s, str) and s.strip()]
    return subs[:5] if subs else [q]

def extract_self(subq: str) -> Dict[str, Any]:
    schema = {
        "bullets": [],
        "metrics": [{"name": "", "value": "", "unit": "", "context": "", "confidence": "low|medium|high"}],
        "claims": [{"statement": "", "confidence": "low|medium|high"}],
        "uncertainties": []
    }
    prompt = (
        "외부 검색 없이, 일반 지식 범위에서 아래 하위 질문의 핵심 정보를 자체적으로 수집·정리해 주세요. "
        "확신도 표기, 불확실/근사는 명시. JSON만 반환.\n\n"
        + json.dumps(schema, ensure_ascii=False)
        + "\n\n하위 질문:\n" + subq
        + f"\n\n지식 컷오프: {KNOWLEDGE_CUTOFF}"
    )
    out = llm_x.invoke([
        SystemMessage(content=STYLE_SYS),
        SystemMessage(content="수치·단위·기호 보존, 불확실성 명시, JSON only."),
        HumanMessage(content=prompt)
    ]).content
    return jload(out, {"bullets": [], "metrics": [], "claims": [], "uncertainties": []})

def aggregate(parts: List[Dict[str, Any]]) -> Dict[str, Any]:
    bullets, seen = [], set()
    metrics, claims, uncertainties = [], [], []
    for p in parts:
        for b in p.get("bullets", []):
            s = b.strip()
            if s and s not in seen:
                seen.add(s); bullets.append(s)
        metrics.extend([m for m in p.get("metrics", []) if isinstance(m, dict)])
        claims.extend([c for c in p.get("claims", []) if isinstance(c, dict)])
        uncertainties.extend([u for u in p.get("uncertainties", []) if isinstance(u, str)])
    return {
        "bullets": bullets[:14],
        "metrics": metrics[:12],
        "claims": claims[:12],
        "uncertainties": uncertainties[:8]
    }

def summarize(query: str, agg: Dict[str, Any]) -> str:
    prompt = (
        "다음 정보를 바탕으로 카드형 요약을 작성해 주세요.\n"
        "- 섹션 3~6개, 각 3~5줄, 친절하고 전문적인 존댓말\n"
        "- 단위/수치/기호 보존, 추정/불확실 명시\n"
        f"- 지식 컷오프: {KNOWLEDGE_CUTOFF}\n\n"
        f"질문: {query}\n"
        f"핵심포인트: {json.dumps(agg.get('bullets', []), ensure_ascii=False)}\n"
        f"지표: {json.dumps(agg.get('metrics', []), ensure_ascii=False)}\n"
        f"주장: {json.dumps(agg.get('claims', []), ensure_ascii=False)}\n"
        f"불확실: {json.dumps(agg.get('uncertainties', []), ensure_ascii=False)}\n"
    )
    return llm_s.invoke([
        SystemMessage(content=STYLE_SYS),
        SystemMessage(content="간결·정확·근거 수준 표시"),
        HumanMessage(content=prompt)
    ]).content

def run(query: str) -> str:
    if not query.strip():
        return "질문을 입력해 주세요."
    # ▶ NEW: 먼저 답변 가능성 평가
    ev = evaluate_answerability(query)
    if not ev.get("answerable", True):
        return guidance_message(ev)
    # ▶ 가능하면 기존 파이프라인 수행
    subs = make_subquestions(ev.get("rewrite_examples",[query])[0] if ev.get("rewrite_examples") else query)
    parts = [extract_self(s) for s in subs]
    agg = aggregate(parts)
    return summarize(query, agg)

def _chat_handler(user_text: str, history: list[list[str]]) -> str:
    # 멀티턴 문맥을 쓰지 않고, 기존 단일턴 파이프라인(run) 그대로 호출
    return run(user_text)

demo = gr.ChatInterface(
    fn=_chat_handler,
    chatbot=gr.Chatbot(height=520, show_copy_button=True),
    title="전력·에너지 요약 챗봇",
    description="시장/산업/업계 동향 관련 질문에 답합니다.",
    examples=[
        "한국 재생에너지 보급 동향 요약",
        "BESS 시장 트렌드 핵심 포인트",
        "EV 충전 인프라 최근 이슈 정리",
    ],
)

demo.launch(share=True)  # Colab은 share=True 필요

/tmp/ipython-input-3431024483.py:195: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=520, show_copy_button=True),
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9aa7893cb3c115a885.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


더 이쁜 UI(최종)

In [ ]:
# API키 및 모델 설정
import os
import json
from typing import Any, Dict, List
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import gradio as gr

os.environ["OPENAI_API_KEY"] = ""
OPENAI_MODEL = "gpt-4o-mini"

MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
llm_g = ChatOpenAI(model=MODEL, temperature=0)     # 가드/판정/리라이트
llm_x = ChatOpenAI(model=MODEL, temperature=0.2)   # 자체 추출
llm_s = ChatOpenAI(model=MODEL, temperature=0.2)   # 요약

KNOWLEDGE_CUTOFF = "2024-06"

STYLE_SYS = (
    "한국어 존댓말을 사용합니다. 말투는 친절하고 전문적으로 유지합니다. "
    "핵심은 간결하게 전달하되, 전력·에너지·모빌리티 분야의 수치/단위/기호(η, THD, pf, pu, kW, kWh, °C 등)는 보존합니다. "
    "불확실하거나 기억이 모호한 내용은 '불확실'로 표시하고 추정·일반론은 명확히 구분합니다. 과장 표현은 지양합니다."
)

ALLOW = [
    "전력","에너지","모빌리티","EV","충전","배터리","신재생","태양광","풍력","수소",
    "인버터","컨버터","PCS","BESS","V2G","EMS","DER","DR","스마트그리드","송배전","변전",
    "THD","pf","pu","고조파","전압안정도","신뢰도","시장","트렌드","동향"
]

def jload(s: str, default: Any) -> Any:
    try:
        return json.loads(s)
    except:
        return default

def evaluate_answerability(q: str) -> Dict[str, Any]:
    """
    JSON 예시
    {
      "answerable": true|false,
      "reasons": ["off_topic","requires_web","knowledge_cutoff","ambiguous","too_broad","privacy_sensitive","unsafe"],
      "explain": "왜 바로 답하기 어려운지 한 줄 설명(존댓말)",
      "ask_for": ["누락된 구체 정보 항목들"],
      "rewrite_examples": ["권장 재질문 1","권장 재질문 2"]
    }
    """
    policy = (
        "당신은 질의가 아래 제약에서 답변 가능한지 판정합니다.\n"
        f"- 지식 컷오프: {KNOWLEDGE_CUTOFF}\n"
        "- 외부 웹 검색/실시간 데이터 사용 불가\n"
        "- 도메인: 전력·에너지·모빌리티 중심\n"
        "- 개인식별/민감정보, 불법/위험, 의료·법률·투자 개별 조언은 불가\n"
        "- 모호/과도하게 광범위/정의가 불명확하면 구체화가 필요\n"
        "위 조건을 바탕으로 JSON만 반환하세요."
    )
    schema = {
        "answerable": True,
        "reasons": [],
        "explain": "",
        "ask_for": [],
        "rewrite_examples": []
    }
    prompt = (
        policy + "\n\n"
        "가능하면 사용자가 바로 쓸 수 있는 '재질문 예시'를 1~3개 제안하세요 "
        "(전력·에너지 맥락의 지표/범위/지역/기간/대상 등을 포함). "
        "JSON만 출력. 스키마는 다음과 같음:\n"
        + json.dumps(schema, ensure_ascii=False) + "\n\n"
        f"질문: {q}"
    )
    out = llm_g.invoke([
        SystemMessage(content=STYLE_SYS),
        SystemMessage(content="JSON only."),
        HumanMessage(content=prompt)
    ]).content
    data = jload(out, schema)
    # 휴리스틱 보강: 실시간/최신/가격/주가/날씨 등
    recent_triggers = ["실시간","현재","오늘","방금","지금","최신","주가","환율","날씨","스코어","속보","라이브"]
    if any(t in q for t in recent_triggers):
        data["answerable"] = False
        if "requires_web" not in data["reasons"]:
            data["reasons"].append("requires_web")
        data["explain"] = data.get("explain") or "실시간·최신 데이터는 웹 접근 없이 정확히 안내드리기 어렵습니다."
    # 도메인 체크
    if not any(k.lower() in q.lower() for k in ALLOW):
        if "off_topic" not in data["reasons"]:
            data["reasons"].append("off_topic")
    return data

def guidance_message(evald: Dict[str, Any]) -> str:
    reasons_ko = {
        "off_topic": "준비된 주제 범위를 벗어났습니다.",
        "requires_web": "실시간/웹 검색이 필요한 주제입니다.",
        "knowledge_cutoff": f"지식 컷오프({KNOWLEDGE_CUTOFF}) 이후 정보가 필요합니다.",
        "ambiguous": "질문이 모호합니다.",
        "too_broad": "질문 범위가 너무 넓습니다.",
        "privacy_sensitive": "개인정보·민감정보라서 도와드리기 어렵습니다.",
        "unsafe": "위험/부적절한 요청입니다."
    }
    lines = ["요청하신 내용은 바로 정확히 답변드리기 어렵습니다."]
    if evald.get("explain"):
        lines.append(f"사유: {evald['explain']}")
    if evald.get("reasons"):
        labs = [reasons_ko.get(r, r) for r in evald["reasons"]]
        lines.append("판정: " + ", ".join(labs))
    if evald.get("ask_for"):
        lines.append("\n다음 정보를 알려주시면 더 정확히 도와드릴 수 있습니다:")
        lines += [f"- {a}" for a in evald["ask_for"][:6]]
    if evald.get("rewrite_examples"):
        lines.append("\n이렇게 질문해 보시면 좋습니다:")
        lines += [f"- {r}" for r in evald["rewrite_examples"][:3]]
    return "\n".join(lines)

def make_subquestions(q: str) -> List[str]:
    prompt = (
        "다음 질문을 3~5개의 하위 질문으로 분해하세요. 최신성 이슈가 있으면 '최근 동향 확인'을 포함합니다. "
        "JSON만 출력: {\"subs\": [\"...\"]}\n\n"
        f"질문: {q}"
    )
    out = llm_x.invoke([SystemMessage(content="JSON only."), HumanMessage(content=prompt)]).content
    data = jload(out, {"subs":[q]})
    subs = [s for s in data.get("subs", []) if isinstance(s, str) and s.strip()]
    return subs[:5] if subs else [q]

def extract_self(subq: str) -> Dict[str, Any]:
    schema = {
        "bullets": [],
        "metrics": [{"name": "", "value": "", "unit": "", "context": "", "confidence": "low|medium|high"}],
        "claims": [{"statement": "", "confidence": "low|medium|high"}],
        "uncertainties": []
    }
    prompt = (
        "외부 검색 없이, 일반 지식 범위에서 아래 하위 질문의 핵심 정보를 자체적으로 수집·정리해 주세요. "
        "확신도 표기, 불확실/근사 명시. JSON만 반환.\n\n"
        + json.dumps(schema, ensure_ascii=False)
        + "\n\n하위 질문:\n" + subq
        + f"\n\n지식 컷오프: {KNOWLEDGE_CUTOFF}"
    )
    out = llm_x.invoke([
        SystemMessage(content=STYLE_SYS),
        SystemMessage(content="수치·단위·기호 보존, 불확실성 명시, JSON only."),
        HumanMessage(content=prompt)
    ]).content
    return jload(out, {"bullets": [], "metrics": [], "claims": [], "uncertainties": []})

def aggregate(parts: List[Dict[str, Any]]) -> Dict[str, Any]:
    bullets, seen = [], set()
    metrics, claims, uncertainties = [], [], []
    for p in parts:
        for b in p.get("bullets", []):
            s = b.strip()
            if s and s not in seen:
                seen.add(s); bullets.append(s)
        metrics.extend([m for m in p.get("metrics", []) if isinstance(m, dict)])
        claims.extend([c for c in p.get("claims", []) if isinstance(c, dict)])
        uncertainties.extend([u for u in p.get("uncertainties", []) if isinstance(u, str)])
    return {
        "bullets": bullets[:14],
        "metrics": metrics[:12],
        "claims": claims[:12],
        "uncertainties": uncertainties[:8]
    }

def summarize(query: str, agg: Dict[str, Any]) -> str:
    prompt = (
        "다음 정보를 바탕으로 카드형 요약을 작성해 주세요.\n"
        "- 섹션 3~6개, 각 3~5줄, 친절하고 전문적인 존댓말\n"
        "- 단위/수치/기호 보존, 추정/불확실 명시\n"
        f"- 지식 컷오프: {KNOWLEDGE_CUTOFF}\n\n"
        f"질문: {query}\n"
        f"핵심포인트: {json.dumps(agg.get('bullets', []), ensure_ascii=False)}\n"
        f"지표: {json.dumps(agg.get('metrics', []), ensure_ascii=False)}\n"
        f"주장: {json.dumps(agg.get('claims', []), ensure_ascii=False)}\n"
        f"불확실: {json.dumps(agg.get('uncertainties', []), ensure_ascii=False)}\n"
    )
    return llm_s.invoke([
        SystemMessage(content=STYLE_SYS),
        SystemMessage(content="간결·정확·근거 수준 표시"),
        HumanMessage(content=prompt)
    ]).content

def run(query: str) -> str:
    if not query.strip():
        return "질문을 입력해 주세요."
    # ▶ NEW: 먼저 답변 가능성 평가
    ev = evaluate_answerability(query)
    if not ev.get("answerable", True):
        return guidance_message(ev)
    # ▶ 가능하면 기존 파이프라인 수행
    subs = make_subquestions(ev.get("rewrite_examples",[query])[0] if ev.get("rewrite_examples") else query)
    parts = [extract_self(s) for s in subs]
    agg = aggregate(parts)
    return summarize(query, agg)

def _chat_handler(user_text: str, history: list[list[str]]) -> str:
    # 멀티턴 문맥을 쓰지 않고, 기존 단일턴 파이프라인(run) 그대로 호출
    return run(user_text)

# 그라디오 UI 색감 및 스타일 변경
with gr.Blocks(theme=gr.themes.Soft(), title="전기·전자 요약 챗봇") as demo:
    gr.HTML(
        """
        <div style="text-align: center; max-width: 700px; margin: 0 auto; padding: 20px;">
            <h1 style="color: #007bff; font-size: 2.5em; font-weight: bold; margin-bottom: 5px;">🔌 전기·전자 요약 챗봇 💡</h1>
            <p style="color: #555; font-size: 1.1em;">시장/산업/업계 동향 관련 질문에 답해 드립니다.</p>
        </div>
        """
    )

    gr.ChatInterface(
        fn=_chat_handler,
        chatbot=gr.Chatbot(
            height=520,
            show_copy_button=True,
            render=False, # 상위 gr.Blocks에 직접 렌더링
            layout="panel"
        ),
        textbox=gr.Textbox(
            placeholder="질문을 입력해 주세요. (예: 한국 재생에너지 보급 동향 요약)",
            container=False,
            scale=7,
        ),
        examples=[
            "한국 재생에너지 보급 동향 요약",
            "BESS 시장 트렌드 핵심 포인트",
            "EV 충전 인프라 최근 이슈 정리",
        ],
    )

    demo.launch(share=True) # Colab은 share=True 필요

/tmp/ipython-input-3019489585.py:213: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eef428fccb1c49a2ab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


최종본(2)

In [ ]:
# -*- coding: utf-8 -*-
# 전기·전자 도메인 요약 챗봇 (주제 분류 기반 응답 집중형)

# ▶ 사전 준비 (.env 파일 예시)
# OPENAI_API_KEY=sk-xxxxx
# OPENAI_MODEL=gpt-4o-mini

import os
import json
from typing import Any, Dict, List

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import gradio as gr

# -----------------------------
# 환경 변수 로드 (.env)
# -----------------------------
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY가 설정되어 있지 않습니다. .env 파일을 준비하거나 환경변수를 설정해 주세요.")

OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

# -----------------------------
# LLM 인스턴스
# -----------------------------
llm_g = ChatOpenAI(model=OPENAI_MODEL, temperature=0)       # 가드/판정/리라이트/분류
llm_x = ChatOpenAI(model=OPENAI_MODEL, temperature=0.2)     # 자체 추출
llm_s = ChatOpenAI(model=OPENAI_MODEL, temperature=0.2)     # 요약

# -----------------------------
# 정책/스타일
# -----------------------------
KNOWLEDGE_CUTOFF = "2024-06"

STYLE_SYS = (
    "한국어 존댓말을 사용합니다. 말투는 친절하고 전문적으로 유지합니다. "
    "핵심은 간결하게 전달하되, 전력·에너지·모빌리티 분야의 수치/단위/기호(η, THD, pf, pu, kW, kWh, °C 등)는 보존합니다. "
    "불확실하거나 기억이 모호한 내용은 '불확실'로 표시하고 추정·일반론은 명확히 구분합니다. 과장 표현은 지양합니다."
)

# 허용 도메인(광범위하게 포함, 재생에너지 편향 제거)
ALLOW = [
    "전력","전력시장","전기","전기모터","모터","에너지","모빌리티","EV","충전","배터리","BESS",
    "신재생","태양광","풍력","수소","인버터","컨버터","PCS","V2G","EMS","DER","DR",
    "스마트그리드","송배전","변전","THD","pf","pu","고조파","전압안정도","신뢰도","시장","트렌드","동향"
]

# 답변 주제 라벨
TOPIC_LABELS = [
    "전력시장", "전기모터", "신재생에너지", "EV/충전", "배터리/BESS",
    "스마트그리드", "송배전/변전", "기타"
]

# -----------------------------
# 유틸 함수
# -----------------------------
def jload(s: str, default: Any) -> Any:
    try:
        return json.loads(s)
    except Exception:
        return default

# -----------------------------
# 주제 분류기
# -----------------------------
def classify_topic(q: str) -> Dict[str, Any]:
    """
    질문을 미리 정의한 토픽 중 하나로 분류.
    출력 예:
    {
      "topic": "전력시장",  # TOPIC_LABELS 중 하나
      "subtopics": ["도매시장", "가격체계"],
      "confidence": "high|medium|low",
      "explain": "간단 근거"
    }
    """
    schema = {"topic": "", "subtopics": [], "confidence": "medium", "explain": ""}
    prompt = (
        "다음 질문을 아래 토픽 중 하나로만 분류하고 JSON으로만 답하세요.\n"
        f"가능한 토픽: {TOPIC_LABELS}\n"
        "규칙:\n"
        "- 질문의 핵심을 가장 잘 대표하는 단 하나의 topic을 고르세요.\n"
        "- subtopics는 1~3개, 없으면 빈 배열.\n"
        "- confidence는 high/medium/low 중 하나.\n"
        "- explain에는 간단한 근거를 1줄로.\n\n"
        f"질문: {q}\n\n"
        f"반환 스키마: {json.dumps(schema, ensure_ascii=False)}"
    )
    out = llm_g.invoke([
        SystemMessage(content="JSON only."),
        HumanMessage(content=prompt)
    ]).content
    data = jload(out, schema)
    if data.get("topic") not in TOPIC_LABELS:
        data["topic"] = "기타"
    if not isinstance(data.get("subtopics"), list):
        data["subtopics"] = []
    return data

# -----------------------------
# 파이프라인 단계 1: 답변 가능성 평가
# -----------------------------
def evaluate_answerability(q: str) -> Dict[str, Any]:
    """
    JSON 예시
    {
      "answerable": true|false,
      "reasons": ["off_topic","requires_web","knowledge_cutoff","ambiguous","too_broad","privacy_sensitive","unsafe"],
      "explain": "왜 바로 답하기 어려운지 한 줄 설명(존댓말)",
      "ask_for": ["누락된 구체 정보 항목들"],
      "rewrite_examples": ["권장 재질문 1","권장 재질문 2"]
    }
    """
    policy = (
        "당신은 질의가 아래 제약에서 답변 가능한지 판정합니다.\n"
        f"- 지식 컷오프: {KNOWLEDGE_CUTOFF}\n"
        "- 외부 웹 검색/실시간 데이터 사용 불가\n"
        "- 도메인: 전력·에너지·모빌리티 중심\n"
        "- 개인식별/민감정보, 불법/위험, 의료·법률·투자 개별 조언은 불가\n"
        "- 모호/과도하게 광범위/정의가 불명확하면 구체화가 필요\n"
        "위 조건을 바탕으로 JSON만 반환하세요."
    )
    schema = {
        "answerable": True,
        "reasons": [],
        "explain": "",
        "ask_for": [],
        "rewrite_examples": []
    }
    prompt = (
        policy + "\n\n"
        "가능하면 사용자가 바로 쓸 수 있는 '재질문 예시'를 1~3개 제안하세요 "
        "(전력·에너지 맥락의 지표/범위/지역/기간/대상 등을 포함). "
        "JSON만 출력. 스키마는 다음과 같음:\n"
        + json.dumps(schema, ensure_ascii=False) + "\n\n"
        f"질문: {q}"
    )
    out = llm_g.invoke([
        SystemMessage(content=STYLE_SYS),
        SystemMessage(content="JSON only."),
        HumanMessage(content=prompt)
    ]).content
    data = jload(out, schema)

    # 휴리스틱 보강: 실시간/최신/가격/주가/날씨 등
    recent_triggers = ["실시간","현재","오늘","방금","지금","최신","주가","환율","날씨","스코어","속보","라이브"]
    if any(t in q for t in recent_triggers):
        data["answerable"] = False
        if "requires_web" not in data["reasons"]:
            data["reasons"].append("requires_web")
        data["explain"] = data.get("explain") or "실시간·최신 데이터는 웹 접근 없이 정확히 안내드리기 어렵습니다."

    # 도메인 체크(광범위 포함)
    if not any(k.lower() in q.lower() for k in ALLOW):
        if "off_topic" not in data["reasons"]:
            data["reasons"].append("off_topic")
    return data

# -----------------------------
# 파이프라인 단계 1-보조: 가이드 문구 생성
# -----------------------------
def guidance_message(evald: Dict[str, Any]) -> str:
    reasons_ko = {
        "off_topic": "준비된 주제 범위를 벗어났습니다.",
        "requires_web": "실시간/웹 검색이 필요한 주제입니다.",
        "knowledge_cutoff": f"지식 컷오프({KNOWLEDGE_CUTOFF}) 이후 정보가 필요합니다.",
        "ambiguous": "질문이 모호합니다.",
        "too_broad": "질문 범위가 너무 넓습니다.",
        "privacy_sensitive": "개인정보·민감정보라서 도와드리기 어렵습니다.",
        "unsafe": "위험/부적절한 요청입니다."
    }
    lines = ["요청하신 내용은 바로 정확히 답변드리기 어렵습니다."]
    if evald.get("explain"):
        lines.append(f"사유: {evald['explain']}")
    if evald.get("reasons"):
        labs = [reasons_ko.get(r, r) for r in evald["reasons"]]
        lines.append("판정: " + ", ".join(labs))
    if evald.get("ask_for"):
        lines.append("\n다음 정보를 알려주시면 더 정확히 도와드릴 수 있습니다:")
        lines += [f"- {a}" for a in evald["ask_for"][:6]]
    if evald.get("rewrite_examples"):
        lines.append("\n이렇게 질문해 보시면 좋습니다:")
        lines += [f"- {r}" for r in evald["rewrite_examples"][:3]]
    return "\n".join(lines)

# -----------------------------
# 파이프라인 단계 2: 하위 질문 분해 (주제 고정)
# -----------------------------
def make_subquestions(q: str, topic: str) -> List[str]:
    prompt = (
        "다음 질문을 3~5개의 하위 질문으로 분해하세요.\n"
        f"- 분류된 주제에만 집중하여 작성: {topic}\n"
        "- 최신성 이슈가 있으면 '최근 동향 확인'을 포함합니다.\n"
        'JSON만 출력: {"subs": ["..."]}\n\n'
        f"질문: {q}"
    )
    out = llm_x.invoke([SystemMessage(content="JSON only."), HumanMessage(content=prompt)]).content
    data = jload(out, {"subs":[q]})
    subs = [s for s in data.get("subs", []) if isinstance(s, str) and s.strip()]
    return subs[:5] if subs else [q]

# -----------------------------
# 파이프라인 단계 3: 자체 정보 추출 (주제 고정)
# -----------------------------
def extract_self(subq: str, topic: str, subtopics: List[str]) -> Dict[str, Any]:
    schema = {
        "bullets": [],
        "metrics": [{"name": "", "value": "", "unit": "", "context": "", "confidence": "low|medium|high"}],
        "claims": [{"statement": "", "confidence": "low|medium|high"}],
        "uncertainties": []
    }
    prompt = (
        "외부 검색 없이, 일반 지식 범위에서 아래 하위 질문의 핵심 정보를 자체적으로 수집·정리해 주세요.\n"
        f"- 분류된 주제: {topic}\n"
        f"- 서브토픽(참고): {', '.join(subtopics) if subtopics else '없음'}\n"
        "- 분류된 주제와 무관한 분야(예: 다른 산업/기술)는 언급하지 마세요.\n"
        "- 확신도 표기, 불확실/근사 명시. JSON만 반환.\n\n"
        + json.dumps(schema, ensure_ascii=False)
        + "\n\n하위 질문:\n" + subq
        + f"\n\n지식 컷오프: {KNOWLEDGE_CUTOFF}"
    )
    out = llm_x.invoke([
        SystemMessage(content=STYLE_SYS),
        SystemMessage(content="수치·단위·기호 보존, 불확실성 명시, JSON only."),
        HumanMessage(content=prompt)
    ]).content
    return jload(out, {"bullets": [], "metrics": [], "claims": [], "uncertainties": []})

# -----------------------------
# 파이프라인 단계 4: 통합
# -----------------------------
def aggregate(parts: List[Dict[str, Any]]) -> Dict[str, Any]:
    bullets, seen = [], set()
    metrics, claims, uncertainties = [], [], []
    for p in parts:
        for b in p.get("bullets", []):
            s = b.strip()
            if s and s not in seen:
                seen.add(s); bullets.append(s)
        metrics.extend([m for m in p.get("metrics", []) if isinstance(m, dict)])
        claims.extend([c for c in p.get("claims", []) if isinstance(c, dict)])
        uncertainties.extend([u for u in p.get("uncertainties", []) if isinstance(u, str)])
    return {
        "bullets": bullets[:14],
        "metrics": metrics[:12],
        "claims": claims[:12],
        "uncertainties": uncertainties[:8]
    }

# -----------------------------
# 파이프라인 단계 5: 요약 생성 (주제 고정)
# -----------------------------
def summarize(query: str, agg: Dict[str, Any], topic: str, subtopics: List[str]) -> str:
    topic_rule = (
        f"이번 응답은 반드시 '{topic}' 주제에만 집중해야 합니다. "
        "분류된 주제와 직접적 관련이 없는 타 분야(예: 다른 에너지 기술/산업)는 언급하지 않습니다."
    )
    prompt = (
        "다음 정보를 바탕으로 카드형 요약을 작성해 주세요.\n"
        "- 섹션 3~6개, 각 3~5줄, 친절하고 전문적인 존댓말\n"
        "- 단위/수치/기호 보존, 추정/불확실 명시\n"
        f"- 지식 컷오프: {KNOWLEDGE_CUTOFF}\n"
        f"- 주제 규칙: {topic_rule}\n"
        f"- 서브토픽(참고): {', '.join(subtopics) if subtopics else '없음'}\n\n"
        f"질문: {query}\n"
        f"핵심포인트: {json.dumps(agg.get('bullets', []), ensure_ascii=False)}\n"
        f"지표: {json.dumps(agg.get('metrics', []), ensure_ascii=False)}\n"
        f"주장: {json.dumps(agg.get('claims', []), ensure_ascii=False)}\n"
        f"불확실: {json.dumps(agg.get('uncertainties', []), ensure_ascii=False)}\n"
    )
    return llm_s.invoke([
        SystemMessage(content=STYLE_SYS),
        SystemMessage(content="간결·정확·근거 수준 표시"),
        HumanMessage(content=prompt)
    ]).content

# -----------------------------
# 실행 핸들러
# -----------------------------
def run(query: str) -> str:
    if not query.strip():
        return "질문을 입력해 주세요."
    # 0) 주제 분류
    cls = classify_topic(query)
    topic = cls.get("topic", "기타")
    subtopics = cls.get("subtopics", [])
    # 1) 답변 가능성 평가
    ev = evaluate_answerability(query)
    if not ev.get("answerable", True):
        # 가이드 메시지에 주제 정보 보강
        guide = guidance_message(ev)
        guide += f"\n\n[분류 결과] 주제: {topic}, 신뢰도: {cls.get('confidence','medium')}"
        return guide
    # 2) 하위 질문 분해 (주제 고정)
    base_q = ev.get("rewrite_examples", [query])[0] if ev.get("rewrite_examples") else query
    subs = make_subquestions(base_q, topic)
    # 3) 자체 추출
    parts = [extract_self(s, topic, subtopics) for s in subs]
    # 4) 통합
    agg = aggregate(parts)
    # 5) 요약 생성 (주제 고정)
    return summarize(query, agg, topic, subtopics)

def _chat_handler(user_text: str, history: list[list[str]]) -> str:
    # 멀티턴 문맥을 쓰지 않고, 기존 단일턴 파이프라인(run) 그대로 호출
    return run(user_text)

# -----------------------------
# Gradio UI
# -----------------------------
with gr.Blocks(theme=gr.themes.Soft(), title="전기·전자 요약 챗봇(주제 분류형)") as demo:
    gr.HTML(
        """
        <div style="text-align: center; max-width: 700px; margin: 0 auto; padding: 20px;">
            <h1 style="color: #007bff; font-size: 2.2em; font-weight: bold; margin-bottom: 5px;">🔌 전기·전자 요약 챗봇 💡</h1>
            <p style="color: #555; font-size: 1.02em;">질문을 자동 분류(전력시장/전기모터/신재생/EV·충전/배터리·BESS/스마트그리드/송배전·변전)하여 주제에 맞게 요약합니다.</p>
        </div>
        """
    )

    gr.ChatInterface(
        fn=_chat_handler,
        chatbot=gr.Chatbot(
            height=520,
            show_copy_button=True,
            render=False, # 상위 gr.Blocks에 직접 렌더링
            layout="panel"
        ),
        textbox=gr.Textbox(
            placeholder="질문을 입력해 주세요. (예: 한국 도매 전력시장 가격 결정 구조 요약)",
            container=False,
            scale=7,
        ),
        examples=[
            "한국 도매 전력시장 가격 결정 구조 요약",
            "영구자석 동기모터(PMSM) 고효율 설계 핵심",
            "태양광 LCOE 구성요소와 민감도",
            "DC 급속충전 인프라 과제",
            "BESS 사이클 수명과 열관리 이슈",
            "분산자원 연계형 스마트그리드 요소",
            "송배전망에서 전압안정도 지표 정리",
        ],
    )

    demo.launch(share=True)  # Colab은 share=True 필요


/tmp/ipython-input-3716230287.py:326: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://30f94fa28a429f7662.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
